In [ ]:
import datetime
import glob
import os

import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('nbs.mplstyle')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd

os.makedirs('../plots/', exist_ok=True)

In [ ]:
df = pd.read_csv('../cache/gitlog.csv')
df['date'] = pd.to_datetime(df['date'].apply(
    lambda x: datetime.datetime.fromisoformat(x).date()))
df = df.set_index('date')

## Number of commits per committer

In [ ]:
n_commits = df.groupby('author').size()

fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

_bins = 10 ** np.arange(0, 4+1e-3, 0.2)
ax.hist(n_commits, bins=_bins, 
        histtype='stepfilled', color='#aaaaaa')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('number of Git commits')
ax.set_ylabel('number of Git committers')

ax.plot(_bins, 150*_bins**-0.5, marker='')

xticks = [1, 10, 100, 1000]
ax.set_xticks(xticks)
ax.set_xticklabels([str(tick) for tick in xticks])

yticks = [1, 10, 100]
ax.set_yticks(yticks)
ax.set_yticklabels([str(tick) for tick in yticks])

ax.set_ylim(0.5, 200)
ax.set_xlim(_bins.min(), _bins.max())

ax.text(100, 20, r'$\propto N^{-0.5}$', fontsize=20)

fig.savefig(f'../plots/astropy-commits-committers.png', dpi=300)

## Cumulative number of commits

In [ ]:
date_bins = df.groupby(pd.Grouper(freq='D'))
commits_per_day = date_bins.size()
n_commits = commits_per_day.cumsum()

fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

n_commits = np.cumsum(commits_per_day)
ax.fill_between(n_commits.index, 0, np.array(n_commits))

ax.set_xlabel('year')
ax.set_ylabel('cumulative Git commits')

ax.set_xlim(datetime.datetime(2011, 6, 1), 
            datetime.datetime.now().date())

ax.set_ylim(0, )

fig.savefig(f'../plots/astropy-cumulative-commits.png', dpi=300)

In [ ]:
date_bins = df.groupby(pd.Grouper(freq='M'))
commits_per_month = date_bins.size()

fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

ax.plot(commits_per_month.index, np.array(commits_per_month),
        drawstyle='steps-mid', marker='')

ax.set_xlabel('year')
ax.set_ylabel('Git commits per month')

ax.set_xlim(datetime.datetime(2011, 6, 1), 
            datetime.datetime.now().date())

ax.set_ylim(0, )

fig.savefig(f'../plots/astropy-commits-per-month.png', dpi=300)

## Monthly number of committers

In [ ]:
groups = df.groupby(pd.Grouper(freq='M'))
agg = groups.agg({"author": lambda x: x.nunique()})

fig, ax = plt.subplots(figsize=(8, 6), constrained_layout=True)

ax.plot(agg.index, agg['author'], drawstyle='steps-mid', marker='')
ax.set_xlabel('year')
ax.set_ylabel('Git committers per month')
ax.set_xlim(datetime.datetime(2011, 6, 1), 
            datetime.datetime.now().date())
ax.set_ylim(0, )

fig.savefig(f'../plots/astropy-commiters-per-month.png', dpi=300)